In [2]:
from influxdb_client import InfluxDBClient, Dialect
import os
import pandas as pd
import numpy as np

In [3]:
username = os.environ.get("$INFLUX_USER")
password = os.environ.get("$INFLUX_PWD")
token = f"{username}:{password}"
bucket = "telegraf/two_weeks"
org = os.environ.get("$INFLUX_ORG")

client = InfluxDBClient(url="https://pepsifree-840e8e88.influxcloud.net:8086", token=token, org=org)
# client = InfluxDBClient(url="https://abb8667f1ceef328437143d160f4d485.m.pipedream.net", token=token)

### InfluxDBClient QueryApi was not working for me at this point -- will revisit

In [4]:
query_api = client.query_api()

In [5]:
f = open("get_data2.flux", "r")
query = f.read()


In [6]:
df = query_api.query_data_frame(query)

AttributeError: 'QueryApi' object has no attribute 'query_data_frame'

### Manually downloaded query results and loading csv this for now

In [7]:
df = pd.read_csv("/Users/samdillard/cloud1_usage.csv", header=0)

In [8]:
df.head(6)

table                           _time                        cluster_id  \
0      0            2020-07-20T20:36:00Z  003007073acd48b281a62808e88e6a83   
1      0            2020-07-20T20:36:30Z  003007073acd48b281a62808e88e6a83   
2      0            2020-07-20T20:37:00Z  003007073acd48b281a62808e88e6a83   
3      0            2020-07-20T20:37:30Z  003007073acd48b281a62808e88e6a83   
4      0  2020-07-20T20:37:31.684994632Z  003007073acd48b281a62808e88e6a83   
5      1            2020-07-20T20:36:00Z  003007073acd48b281a62808e88e6a83   

   cpu_load1  cpu_load15  cpu_load5  cpu_usage_system  cpu_usage_user  \
0       0.27        0.18       0.20          0.664962       22.250639   
1       0.28        0.18       0.21          0.528541        4.809725   
2       0.17        0.18       0.19          0.647733        2.989537   
3       0.10        0.17       0.17          0.491642        1.573255   
4       0.08        0.17       0.17          0.649026        1.048427   
5       0.30        0.19       0.20          0.638298        9.680851   

                                host  mem_available      mem_used  n_cpus  \
0  prod-00300707-ca-central-1-data-3   1.239749e+10  6.902960e+09     2.0   
1  prod-00300707-ca-central-1-data-3   1.236135e+10  6.937002e+09     2.0   
2  prod-00300707-ca-central-1-data-3   1.236001e+10  6.936367e+09     2.0   
3  prod-00300707-ca-central-1-data-3   1.235966e+10  6.936715e+09     2.0   
4  prod-00300707-ca-central-1-data-3   1.235953e+10  6.936842e+09     2.0   
5  prod-00300707-ca-central-1-data-4   1.241994e+10  9.830359e+09     2.0   

   point_req_per_sec  queries_per_sec  read_bytes_per_10s  reads_per_10s  \
0                NaN              NaN                   0              0   
1                NaN         0.066667                   0              0   
2                NaN         0.000000                   0              0   
3                NaN         0.066667                   0              0   
4           0.066667         0.000000                   0              0   
5                NaN              NaN                   0              0   

   write_bytes_per_10s  writes_per_10s  
0                73728               3  
1               102400               9  
2                81920              10  
3                28672               3  
4                40960               9  
5               495616              12

In [9]:
df_no_nan = df.dropna()

In [10]:
len(df)


3270

In [11]:
len(df_no_nan)

627

In [43]:
# Remove all but one row per distinct value of `table`.  
# This ensures no autocorrelation as each row per table is along the same series at consecutive points in time
new_df = df_no_nan.drop_duplicates(subset = ["host"])

In [44]:
new_df.head()

table                           _time                        cluster_id  \
4       0  2020-07-20T20:37:31.684994632Z  003007073acd48b281a62808e88e6a83   
9       1  2020-07-20T20:37:31.684994632Z  003007073acd48b281a62808e88e6a83   
12      2  2020-07-20T20:37:31.684994632Z  00edc633aa6ee38931d5af4a03a0996e   
17      3  2020-07-20T20:37:31.684994632Z  00edc633aa6ee38931d5af4a03a0996e   
22      4  2020-07-20T20:37:31.684994632Z  0202e272af128f07d5b2b86f3b59cf9b   

    cpu_load1  cpu_load15  cpu_load5  cpu_usage_system  cpu_usage_user  \
4        0.08        0.17       0.17          0.649026        1.048427   
9        0.05        0.17       0.14          0.262055        2.777778   
12       0.48        0.66       0.56          0.983284       10.816126   
17       0.36        0.62       0.54          1.335388        7.241911   
22       0.14        0.01       0.06          0.363448        1.765317   

                                 host  mem_available      mem_used  n_cpus  \
4   prod-00300707-ca-central-1-data-3   1.235953e+10  6.936842e+09     2.0   
9   prod-00300707-ca-central-1-data-4   1.242032e+10  9.829982e+09     2.0   
12  prod-00edc633-eu-central-1-data-3   9.678406e+09  1.016034e+10     2.0   
17  prod-00edc633-eu-central-1-data-4   8.610247e+09  1.125026e+10     2.0   
22     prod-0202e272-us-west-2-data-3   3.392741e+09  1.416053e+09     2.0   

    point_req_per_sec  queries_per_sec  read_bytes_per_10s  reads_per_10s  \
4            0.066667              0.0                   0              0   
9            0.071429              0.0                   0              0   
12        1121.800000              0.0                   0              0   
17         633.066667              0.0                   0              0   
22         821.875000              0.0                   0              0   

    write_bytes_per_10s  writes_per_10s  
4                 40960               9  
9                 28672               3  
12              3883008             255  
17              2826240             200  
22              2527232             105

In [45]:
new_df.describe()

table  cpu_load1  cpu_load15   cpu_load5  cpu_usage_system  \
count  617.000000  617.00000  617.000000  617.000000        617.000000   
mean   330.283630    0.64222    0.643031    0.644441          0.849480   
std    190.547977    1.32775    1.212552    1.223869          0.811605   
min      0.000000    0.00000    0.000000    0.000000          0.040950   
25%    167.000000    0.01000    0.030000    0.040000          0.384193   
50%    330.000000    0.15000    0.160000    0.180000          0.589681   
75%    493.000000    0.60000    0.640000    0.600000          0.966118   
max    663.000000   12.36000   10.590000   11.320000          7.802981   

       cpu_usage_user  mem_available      mem_used      n_cpus  \
count      617.000000   6.170000e+02  6.170000e+02  617.000000   
mean         6.603361   2.144745e+10  8.653915e+09    4.103728   
std         12.256586   3.736155e+10  1.327756e+10    5.058964   
min          0.232050   5.251891e+08  3.366625e+08    1.000000   
25%          1.080247   3.199361e+09  1.430839e+09    2.000000   
50%          1.925134   6.273520e+09  2.958754e+09    2.000000   
75%          5.943026   1.988793e+10  1.016034e+10    4.000000   
max        112.167180   2.284660e+11  1.127240e+11   32.000000   

       point_req_per_sec  queries_per_sec  read_bytes_per_10s  reads_per_10s  \
count         617.000000       617.000000        6.170000e+02     617.000000   
mean         1629.433760       372.944289        5.220481e+06      58.829822   
std          4125.308923      4951.898939        4.509746e+07     466.427378   
min             0.000000         0.000000        0.000000e+00       0.000000   
25%             0.600000         0.000000        0.000000e+00       0.000000   
50%            97.333333         0.000000        0.000000e+00       0.000000   
75%          1069.533333         2.373895        0.000000e+00       0.000000   
max         24801.066670    112525.433300        5.416264e+08    4970.000000   

       write_bytes_per_10s  writes_per_10s  
count         6.170000e+02      617.000000  
mean          7.201182e+07      948.359806  
std           2.642215e+08     2444.488173  
min           0.000000e+00        0.000000  
25%           6.553600e+04        8.000000  
50%           8.273920e+05       61.000000  
75%           5.844992e+06      404.000000  
max           2.508063e+09    19323.000000

In [46]:
# Pearson correlation between columns
correlations = new_df.corr()
correlations

table  cpu_load1  cpu_load15  cpu_load5  \
table                1.000000  -0.034055   -0.010750  -0.015191   
cpu_load1           -0.034055   1.000000    0.945406   0.968687   
cpu_load15          -0.010750   0.945406    1.000000   0.992435   
cpu_load5           -0.015191   0.968687    0.992435   1.000000   
cpu_usage_system    -0.024508   0.427448    0.478999   0.468411   
cpu_usage_user      -0.030801   0.536112    0.571677   0.562784   
mem_available       -0.001536   0.660810    0.639904   0.642039   
mem_used            -0.038744   0.699713    0.693597   0.694060   
n_cpus              -0.024544   0.676857    0.655007   0.657206   
point_req_per_sec   -0.016265   0.708581    0.711314   0.716905   
queries_per_sec     -0.062116   0.012754    0.019962   0.016177   
read_bytes_per_10s   0.051407   0.114405    0.143726   0.125323   
reads_per_10s        0.041186   0.115669    0.145294   0.127614   
write_bytes_per_10s -0.055846   0.566754    0.596411   0.588443   
writes_per_10s      -0.057059   0.699581    0.720994   0.713939   

                     cpu_usage_system  cpu_usage_user  mem_available  \
table                       -0.024508       -0.030801      -0.001536   
cpu_load1                    0.427448        0.536112       0.660810   
cpu_load15                   0.478999        0.571677       0.639904   
cpu_load5                    0.468411        0.562784       0.642039   
cpu_usage_system             1.000000        0.697377      -0.000363   
cpu_usage_user               0.697377        1.000000       0.192216   
mem_available               -0.000363        0.192216       1.000000   
mem_used                     0.164404        0.329188       0.798856   
n_cpus                       0.004195        0.190428       0.988714   
point_req_per_sec            0.317624        0.392049       0.572018   
queries_per_sec              0.185997        0.320906      -0.019567   
read_bytes_per_10s           0.132041        0.138394       0.111955   
reads_per_10s                0.125483        0.135439       0.109125   
write_bytes_per_10s          0.231731        0.281546       0.618426   
writes_per_10s               0.379473        0.367348       0.659567   

                     mem_used    n_cpus  point_req_per_sec  queries_per_sec  \
table               -0.038744 -0.024544          -0.016265        -0.062116   
cpu_load1            0.699713  0.676857           0.708581         0.012754   
cpu_load15           0.693597  0.655007           0.711314         0.019962   
cpu_load5            0.694060  0.657206           0.716905         0.016177   
cpu_usage_system     0.164404  0.004195           0.317624         0.185997   
cpu_usage_user       0.329188  0.190428           0.392049         0.320906   
mem_available        0.798856  0.988714           0.572018        -0.019567   
mem_used             1.000000  0.819047           0.618014        -0.008954   
n_cpus               0.819047  1.000000           0.580910        -0.018392   
point_req_per_sec    0.618014  0.580910           1.000000         0.032654   
queries_per_sec     -0.008954 -0.018392           0.032654         1.000000   
read_bytes_per_10s   0.063488  0.114318           0.240757         0.000763   
reads_per_10s        0.072969  0.112980           0.227327        -0.002177   
write_bytes_per_10s  0.508900  0.641447           0.478120         0.010948   
writes_per_10s       0.598384  0.684839           0.631280         0.006213   

                     read_bytes_per_10s  reads_per_10s  write_bytes_per_10s  \
table                          0.051407       0.041186            -0.055846   
cpu_load1                      0.114405       0.115669             0.566754   
cpu_load15                     0.143726       0.145294             0.596411   
cpu_load5                      0.125323       0.127614             0.588443   
cpu_usage_system               0.132041       0.125483             0.231731   
cpu_usage_user                 0.138394       0.135439     

In [47]:
correlations.sort_values(['cpu_load1','cpu_load5','cpu_load15'], ascending=False)

table  cpu_load1  cpu_load15  cpu_load5  \
cpu_load1           -0.034055   1.000000    0.945406   0.968687   
cpu_load5           -0.015191   0.968687    0.992435   1.000000   
cpu_load15          -0.010750   0.945406    1.000000   0.992435   
point_req_per_sec   -0.016265   0.708581    0.711314   0.716905   
mem_used            -0.038744   0.699713    0.693597   0.694060   
writes_per_10s      -0.057059   0.699581    0.720994   0.713939   
n_cpus              -0.024544   0.676857    0.655007   0.657206   
mem_available       -0.001536   0.660810    0.639904   0.642039   
write_bytes_per_10s -0.055846   0.566754    0.596411   0.588443   
cpu_usage_user      -0.030801   0.536112    0.571677   0.562784   
cpu_usage_system    -0.024508   0.427448    0.478999   0.468411   
reads_per_10s        0.041186   0.115669    0.145294   0.127614   
read_bytes_per_10s   0.051407   0.114405    0.143726   0.125323   
queries_per_sec     -0.062116   0.012754    0.019962   0.016177   
table                1.000000  -0.034055   -0.010750  -0.015191   

                     cpu_usage_system  cpu_usage_user  mem_available  \
cpu_load1                    0.427448        0.536112       0.660810   
cpu_load5                    0.468411        0.562784       0.642039   
cpu_load15                   0.478999        0.571677       0.639904   
point_req_per_sec            0.317624        0.392049       0.572018   
mem_used                     0.164404        0.329188       0.798856   
writes_per_10s               0.379473        0.367348       0.659567   
n_cpus                       0.004195        0.190428       0.988714   
mem_available               -0.000363        0.192216       1.000000   
write_bytes_per_10s          0.231731        0.281546       0.618426   
cpu_usage_user               0.697377        1.000000       0.192216   
cpu_usage_system             1.000000        0.697377      -0.000363   
reads_per_10s                0.125483        0.135439       0.109125   
read_bytes_per_10s           0.132041        0.138394       0.111955   
queries_per_sec              0.185997        0.320906      -0.019567   
table                       -0.024508       -0.030801      -0.001536   

                     mem_used    n_cpus  point_req_per_sec  queries_per_sec  \
cpu_load1            0.699713  0.676857           0.708581         0.012754   
cpu_load5            0.694060  0.657206           0.716905         0.016177   
cpu_load15           0.693597  0.655007           0.711314         0.019962   
point_req_per_sec    0.618014  0.580910           1.000000         0.032654   
mem_used             1.000000  0.819047           0.618014        -0.008954   
writes_per_10s       0.598384  0.684839           0.631280         0.006213   
n_cpus               0.819047  1.000000           0.580910        -0.018392   
mem_available        0.798856  0.988714           0.572018        -0.019567   
write_bytes_per_10s  0.508900  0.641447           0.478120         0.010948   
cpu_usage_user       0.329188  0.190428           0.392049         0.320906   
cpu_usage_system     0.164404  0.004195           0.317624         0.185997   
reads_per_10s        0.072969  0.112980           0.227327        -0.002177   
read_bytes_per_10s   0.063488  0.114318           0.240757         0.000763   
queries_per_sec     -0.008954 -0.018392           0.032654         1.000000   
table               -0.038744 -0.024544          -0.016265        -0.062116   

                     read_bytes_per_10s  reads_per_10s  write_bytes_per_10s  \
cpu_load1                      0.114405       0.115669             0.566754   
cpu_load5                      0.125323       0.127614             0.588443   
cpu_load15                     0.143726       0.145294             0.596411   
point_req_per_sec              0.240757       0.227327             0.478120   
mem_used                       0.063488       0.072969             0.508900   
writes_per_10s                 0.200798       0.187412     

In [48]:
# Multiple `_10s` columns' values by 10 so they can match the `per_sec` clolumns
new_df[['write_bytes_per_10s','writes_per_10s','read_bytes_per_10s','reads_per_10s']] = new_df[['write_bytes_per_10s','writes_per_10s','read_bytes_per_10s','reads_per_10s']].apply(lambda x: x * 10)

In [55]:
new_df.rename(columns={'write_bytes_per_10s':'write_bytes_per_sec', 'writes_per_10s':'writes_per_sec','read_bytes_per_10s':'read_bytes_per_sec','reads_per_10s':'reads_per_sec'})

table                           _time                        cluster_id  \
4         0  2020-07-20T20:37:31.684994632Z  003007073acd48b281a62808e88e6a83   
9         1  2020-07-20T20:37:31.684994632Z  003007073acd48b281a62808e88e6a83   
12        2  2020-07-20T20:37:31.684994632Z  00edc633aa6ee38931d5af4a03a0996e   
17        3  2020-07-20T20:37:31.684994632Z  00edc633aa6ee38931d5af4a03a0996e   
22        4  2020-07-20T20:37:31.684994632Z  0202e272af128f07d5b2b86f3b59cf9b   
...     ...                             ...                               ...   
3249    659  2020-07-20T20:37:31.684994632Z  fec5652ef93b88bd3b14421270758144   
3254    660  2020-07-20T20:37:31.684994632Z  ff2aeacdc4b196013d9c4da0fc441f1c   
3259    661  2020-07-20T20:37:31.684994632Z  ff2aeacdc4b196013d9c4da0fc441f1c   
3264    662  2020-07-20T20:37:31.684994632Z  fffe53a334d5fe4d994e1d8a65b0b761   
3269    663  2020-07-20T20:37:31.684994632Z  fffe53a334d5fe4d994e1d8a65b0b761   

      cpu_load1  cpu_load15  cpu_load5  cpu_usage_system  cpu_usage_user  \
4          0.08        0.17       0.17          0.649026        1.048427   
9          0.05        0.17       0.14          0.262055        2.777778   
12         0.48        0.66       0.56          0.983284       10.816126   
17         0.36        0.62       0.54          1.335388        7.241911   
22         0.14        0.01       0.06          0.363448        1.765317   
...         ...         ...        ...               ...             ...   
3249       0.48        0.22       0.30          0.518967       16.458668   
3254       0.00        0.00       0.00          0.408998        0.817996   
3259       0.00        0.00       0.00          0.530632        0.868307   
3264       0.29        0.22       0.24          0.320856        1.657754   
3269       0.08        0.31       0.29          0.481580        2.672767   

                                   host  mem_available      mem_used  n_cpus  \
4     prod-00300707-ca-central-1-data-3   1.235953e+10  6.936842e+09     2.0   
9     prod-00300707-ca-central-1-data-4   1.242032e+10  9.829982e+09     2.0   
12    prod-00edc633-eu-central-1-data-3   9.678406e+09  1.016034e+10     2.0   
17    prod-00edc633-eu-central-1-data-4   8.610247e+09  1.125026e+10     2.0   
22       prod-0202e272-us-west-2-data-3   3.392741e+09  1.416053e+09     2.0   
...                                 ...            ...           ...     ...   
3249     prod-fec5652e-eu-west-1-data-6   4.722816e+10  1.636107e+10     8.0   
3254     prod-ff2aeacd-us-west-2-data-3   2.705551e+09  2.547298e+09     2.0   
3259     prod-ff2aeacd-us-west-2-data-4   2.692952e+09  2.515997e+09     2.0   
3264     prod-fffe53a3-us-east-1-data-3   2.876565e+10  8.237044e+09     4.0   
3269     prod-fffe53a3-us-east-1-data-4   2.855392e+10  7.336124e+09     4.0   

      point_req_per_sec  queries_per_sec  read_bytes_per_sec  reads_per_sec  \
4              0.066667         0.000000                   0              0   
9              0.071429         0.000000                   0              0   
12          1121.800000         0.000000                   0              0   
17           633.066667         0.000000                   0              0   
22           821.875000         0.000000                   0              0   
...                 ...              ...                 ...            ...   
3249        1566.400000         8.902106                   0              0   
3254           0.000000         0.000000                   0              0   
3259           0.000000         0.000000                   0              0   
3264         166.600000         0.000000                   0              0   
3269          37.066667         0.593474                   0              0   

      write_bytes_per_sec  writes_per_sec  
4                  409600              90  
9                  286720              30  
12               38830080            2550  
17               28262400         

In [56]:
# Get new correlation matrix with the above modification
# This matrix shows the most correlated metrics with `load` values (exclude other load metrics)
correlations = new_df.corr()
correlations.sort_values(['cpu_load1','cpu_load5','cpu_load15'], ascending=False)

table  cpu_load1  cpu_load15  cpu_load5  \
cpu_load1           -0.034055   1.000000    0.945406   0.968687   
cpu_load5           -0.015191   0.968687    0.992435   1.000000   
cpu_load15          -0.010750   0.945406    1.000000   0.992435   
point_req_per_sec   -0.016265   0.708581    0.711314   0.716905   
mem_used            -0.038744   0.699713    0.693597   0.694060   
writes_per_10s      -0.057059   0.699581    0.720994   0.713939   
n_cpus              -0.024544   0.676857    0.655007   0.657206   
mem_available       -0.001536   0.660810    0.639904   0.642039   
write_bytes_per_10s -0.055846   0.566754    0.596411   0.588443   
cpu_usage_user      -0.030801   0.536112    0.571677   0.562784   
cpu_usage_system    -0.024508   0.427448    0.478999   0.468411   
reads_per_10s        0.041186   0.115669    0.145294   0.127614   
read_bytes_per_10s   0.051407   0.114405    0.143726   0.125323   
queries_per_sec     -0.062116   0.012754    0.019962   0.016177   
table                1.000000  -0.034055   -0.010750  -0.015191   

                     cpu_usage_system  cpu_usage_user  mem_available  \
cpu_load1                    0.427448        0.536112       0.660810   
cpu_load5                    0.468411        0.562784       0.642039   
cpu_load15                   0.478999        0.571677       0.639904   
point_req_per_sec            0.317624        0.392049       0.572018   
mem_used                     0.164404        0.329188       0.798856   
writes_per_10s               0.379473        0.367348       0.659567   
n_cpus                       0.004195        0.190428       0.988714   
mem_available               -0.000363        0.192216       1.000000   
write_bytes_per_10s          0.231731        0.281546       0.618426   
cpu_usage_user               0.697377        1.000000       0.192216   
cpu_usage_system             1.000000        0.697377      -0.000363   
reads_per_10s                0.125483        0.135439       0.109125   
read_bytes_per_10s           0.132041        0.138394       0.111955   
queries_per_sec              0.185997        0.320906      -0.019567   
table                       -0.024508       -0.030801      -0.001536   

                     mem_used    n_cpus  point_req_per_sec  queries_per_sec  \
cpu_load1            0.699713  0.676857           0.708581         0.012754   
cpu_load5            0.694060  0.657206           0.716905         0.016177   
cpu_load15           0.693597  0.655007           0.711314         0.019962   
point_req_per_sec    0.618014  0.580910           1.000000         0.032654   
mem_used             1.000000  0.819047           0.618014        -0.008954   
writes_per_10s       0.598384  0.684839           0.631280         0.006213   
n_cpus               0.819047  1.000000           0.580910        -0.018392   
mem_available        0.798856  0.988714           0.572018        -0.019567   
write_bytes_per_10s  0.508900  0.641447           0.478120         0.010948   
cpu_usage_user       0.329188  0.190428           0.392049         0.320906   
cpu_usage_system     0.164404  0.004195           0.317624         0.185997   
reads_per_10s        0.072969  0.112980           0.227327        -0.002177   
read_bytes_per_10s   0.063488  0.114318           0.240757         0.000763   
queries_per_sec     -0.008954 -0.018392           0.032654         1.000000   
table               -0.038744 -0.024544          -0.016265        -0.062116   

                     read_bytes_per_10s  reads_per_10s  write_bytes_per_10s  \
cpu_load1                      0.114405       0.115669             0.566754   
cpu_load5                      0.125323       0.127614             0.588443   
cpu_load15                     0.143726       0.145294             0.596411   
point_req_per_sec              0.240757       0.227327             0.478120   
mem_used                       0.063488       0.072969             0.508900   
writes_per_10s                 0.200798       0.187412     

In [59]:
# Will first try linear regression using SciKitLearn library
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [63]:
new_df.columns = ['table', '_time', 'cluster_id', 'cpu_load1', 'cpu_load15', 'cpu_load5',
       'cpu_usage_system', 'cpu_usage_user', 'host', 'mem_available',
       'mem_used', 'n_cpus', 'point_req_per_sec', 'queries_per_sec',
       'read_bytes_per_sec', 'reads_per_sec', 'write_bytes_per_sec',
       'writes_per_sec']

In [64]:
X = new_df[['point_req_per_sec','queries_per_sec','reads_per_sec','read_bytes_per_sec','writes_per_sec','write_bytes_per_sec']]

In [66]:
y = new_df[['cpu_load1']]

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [71]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((493, 6), (124, 6), (493, 1), (124, 1))

In [73]:
reg = linear_model.LinearRegression()
reg

LinearRegression()

In [74]:
reg.fit(X_train,y_train)

LinearRegression()

In [75]:
reg.coef_

array([[ 1.22109462e-04,  2.27818162e-07,  5.19731782e-05,
        -7.63943712e-10,  3.79539346e-05, -1.35030258e-10]])

In [78]:
reg.

array([0.16274385])

In [77]:
reg.score(X_test,y_test)

0.5303084585346984